# 자동 생성 필터

## 개요

이 노트북은 Amazon Bedrock Knowledge Bases의 **Autogenerated Filters** 기능을 시연합니다. 이 기능은 메타데이터를 자동으로 필터링하여 검색 기능을 향상시킵니다. Amazon Bedrock의 Foundation Model을 활용해 Autogenerated Filters는 사용자 질의를 동적으로 해석하고 Knowledge Base에 정의된 메타데이터 스키마를 기준으로 적절한 메타데이터 필터를 생성합니다. 별도의 필터를 명시하지 않아도 검색 결과의 품질과 관련성을 높일 수 있습니다.

Autogenerated Filters 기능은 `Retrieve` 및 `RetrieveAndGenerate` API 호출의 `retrievalConfiguration` 내부 `vectorSearchConfiguration`에 있는 `implicitFilterConfiguration` 매개변수를 통해 활성화됩니다. 이 API들은 사용자 질의를 분석하여 지정된 `implicitFilterConfiguration`에 따라 관련 메타데이터 속성을 식별하고, 해당 필터를 적용해 검색 결과를 좁힙니다.

예를 들어 사용자가 "지난해의 마케팅 보고서"를 검색하면, Autogenerated Filters 기능은 "지난해"가 특정 기간을 의미함을 자동으로 인식하고 날짜 메타데이터 필드에 기반한 필터를 적용할 수 있습니다. 질의에 특정 제품이나 부서가 언급된 경우에도 시스템은 해당 메타데이터 필드를 바탕으로 필터를 적용합니다.

이제 Amazon Bedrock Knowledge Bases에서 Autogenerated Filters를 구현하고 활용하는 방법을 예제 시나리오를 통해 살펴보겠습니다.

<div class="alert alert-block alert-warning">
<b>참고:</b> 이 노트북을 SageMaker 환경에서 실행하는 경우, SageMaker 실행 역할에 `CloudWatchLogsReadOnlyAccess` 정책과 필요한 기타 접근 권한이 있는지 확인하세요.
</div>

## 1. 설정
나머지 셀을 실행하기 전에 아래 셀을 먼저 실행해 필요한 라이브러리를 설치하고 Bedrock에 연결하세요.

라이브러리를 설치하는 동안 pip 의존성 오류가 표시되더라도 무시해도 됩니다.

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

In [1]:
%pip install --upgrade boto3

  Using cached botocore-1.37.7-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.37.7-py3-none-any.whl (13.4 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.37.5
    Uninstalling botocore-1.37.5:
      Successfully uninstalled botocore-1.37.5
  Attempting uninstall: boto3
    Found existing installation: boto3 1.37.5
    Uninstalling boto3-1.37.5:
      Successfully uninstalled boto3-1.37.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.15.2 requires botocore<1.35.37,>=1.35.16, but you have botocore 1.37.7 which is incompatible.
awscli 1.38.5 requires botocore==1.37.5, but you have botocore 1.37.7 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import boto3
print(boto3.__version__)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [ ]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

In [ ]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"
knowledge_base_name = 'autogenerated-filters-kb'
knowledge_base_description = "Knowledge Base autogenerated metadata filtering."
bucket_name = f'{knowledge_base_name}-{suffix}'
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

# Define data sources
data_source=[{"type": "S3", "bucket_name": bucket_name}]

## 2 - 고정 청킹 전략으로 Knowledge Base 생성
비디오 게임 데이터를 CSV 형식으로 저장할 [Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/)를 먼저 생성해 보겠습니다. Knowledge Bases는 [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/), [Amazon Aurora](https://aws.amazon.com/rds/aurora/), [Pinecone](http://app.pinecone.io/bedrock-integration), [Redis Enterprise]() 및 [MongoDB Atlas]() 등 다양한 벡터 데이터베이스와 통합할 수 있습니다. 이 예제에서는 Knowledge Base를 Amazon OpenSearch Serverless와 연동합니다. 이를 위해 Knowledge Base와 필요한 사전 준비 작업을 모두 생성해 주는 헬퍼 클래스 `BedrockKnowledgeBase`를 사용합니다:
1. IAM 역할 및 정책
2. S3 버킷
3. Amazon OpenSearch Serverless 암호화, 네트워크 및 데이터 접근 정책
4. Amazon OpenSearch Serverless 컬렉션
5. Amazon OpenSearch Serverless 벡터 인덱스
6. Knowledge Base
7. Knowledge Base 데이터 소스

고정 청킹 전략을 사용해 Knowledge Base를 생성합니다. 

아래 매개변수 값을 변경하면 다른 청킹 전략을 선택할 수 있습니다: 
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [ ]:
knowledge_base_metadata = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source, 
    chunking_strategy = "FIXED_SIZE", 
    suffix = suffix
)

### 2.1 Amazon 2019, 2020, 2021, 2022, 2023 연차보고서를 다운로드해 Amazon S3에 업로드하기

Knowledge Base를 만들었으니 `sec-10-k reports` 데이터 세트로 내용을 채워 보겠습니다. 이 데이터는 [여기](https://ir.aboutamazon.com/annual-reports-proxies-and-shareholder-letters/default.aspx)에서 다운로드하며, Amazon의 연차보고서, 위임장, 주주 서한에 대한 자료입니다.

In [ ]:
import os

def create_directory(directory_name):    
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    else:
        print(f"Directory '{directory_name}' already exists.")

# Call the function to create the directory
create_directory("sec-10-k")

In [ ]:
import requests

def download_file(url, filename):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Open the file in write-binary mode
        with open(filename, 'wb') as file:
            # Write the content of the response to the file
            file.write(response.content)
        print(f"File downloaded successfully: {filename}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

# URL of the files to download
urls = ["https://s2.q4cdn.com/299287126/files/doc_financials/2024/ar/Amazon-com-Inc-2023-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/Amazon-2022-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/Amazon-2021-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Annual-Report.pdf"]


for url in urls:
    # Name for the downloaded file
    filename = url.split('/')[-1]

    # Path to save the downloaded file
    filepath = f"./sec-10-k/{filename}"

    # Call the function to download the file
    download_file(url, filepath)

`sec-10-k` 폴더에 있는 연차보고서 데이터를 S3에 업로드합니다.

In [ ]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                if not file.startswith('.DS_Store'):
                    file_to_upload = os.path.join(root,file)
                    print(f"uploading file {file_to_upload} to {bucket_name}")
                    s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("sec-10-k", bucket_name)

#### 2.3 적재용 메타데이터 준비

In [ ]:
import json
import re

def generate_matadata(data_dir):
    
    # Loop through all PDF files in the directory
    for filename in os.listdir(data_dir):
        if not filename.startswith('.DS_Store'):
            # Define the metadata dictionary
            metadata ={}
            
            filename= f'{data_dir}/{filename}'
            print(filename)
            
            # Create metadata
            metadata["company"] = "Amazon"
            metadata["ticker"] = "AMZN"
            metadata["year"] = re.search(r'\d+', filename.split('/')[-1]).group(0)

            # Create a JSON object
            json_data = {"metadataAttributes": metadata}

            # print(json_data)

            # Write the JSON object to a file
            with open(f"{filename.replace('.pdf', '.pdf.metadata.json')}", "w") as f:
                json.dump(json_data, f)


In [ ]:
data_dir = './sec-10-k'
generate_matadata(data_dir)

In [ ]:
# upload metadata file to S3
upload_directory("sec-10-k", bucket_name)

이제 ingestion 작업을 시작합니다. 이미 고정 청킹에서 사용한 것과 동일한 문서를 사용하므로, S3 버킷에 문서를 업로드하는 단계는 건너뜁니다.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_metadata.start_ingestion_job()

마지막으로 나중에 솔루션을 테스트할 수 있도록 Knowledge Base ID를 저장합니다.

In [ ]:
kb_id_metadata = knowledge_base_metadata.get_knowledge_base_id()

### 2.4 Autogenerated Filters 디버깅을 위한 CloudWatch 로그 활성화

아래에서는 디버깅을 위해 CloudWatch 로그를 활성화하는 헬퍼 함수를 만듭니다.

#### CloudWatch 로그용 헬퍼 함수

이 헬퍼 함수들은 CloudWatch 로그를 조회하고 처리하여 자동 생성된 필터가 어떻게 만들어졌는지 분석하는 데 도움이 됩니다. 사용자 질의가 메타데이터 필터 생성에 어떻게 활용되는지에 대한 통찰을 제공합니다. 

이 함수들을 사용하면 실제로 생성된 필터를 확인하고, 다양한 사용자 질의 변화에서도 일관되게 동작하는지 검증하며, 기대하는 동작과 질의 로직에 맞는지 확인할 수 있습니다. 필터가 일관되지 않거나 예상과 다르게 생성되면, 이러한 함수로 원인을 파악하고 문제를 해결할 수 있습니다.

In [ ]:
import boto3
from botocore.exceptions import ClientError

def create_cloudwatch_log_group_for_bedrock(log_group_name):
    logs_client = boto3.client('logs')
    try:
        logs_client.create_log_group(logGroupName=log_group_name)
        print(f"Successfully created CloudWatch log group: {log_group_name}")
        return True
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceAlreadyExistsException':
            print(f"Log group {log_group_name} already exists.")
            return True
        else:
            print(f"Error creating log group: {e}")
            return False

def enable_bedrock_invokemodel_logs(cw_log_group_name):
    bedrock_client = boto3.client('bedrock')
    log_group_name = cw_log_group_name
    
    if not create_cloudwatch_log_group_for_bedrock(log_group_name):
        print("Failed to create or confirm log group. Aborting log enablement.")
        return False

    try:
        get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb_id_metadata)
        role_arn = get_kb_response['knowledgeBase']['roleArn']
        kb_role_name = role_arn.split('/')[-1]

        sts_client = boto3.client('sts')
        account_id = sts_client.get_caller_identity()["Account"]
        role_name = kb_role_name  
        role_arn = f"arn:aws:iam::{account_id}:role/{role_name}"

        # Get all available model types
        response = bedrock_client.list_foundation_models()
        all_model_types = list(set([model['modelName'] for model in response['modelSummaries']]))

        response = bedrock_client.put_model_invocation_logging_configuration(
            loggingConfig={
                'cloudWatchConfig': {
                    'logGroupName': log_group_name,
                    'roleArn': role_arn
                },
                'textDataDeliveryEnabled': True,
                'imageDataDeliveryEnabled': True,
                'embeddingDataDeliveryEnabled': True,
                'videoDataDeliveryEnabled': True
            }
        )
        
        print(f"Successfully enabled InvokeModel logging for Bedrock with log group: {log_group_name}")
        return True
    except ClientError as e:
        print(f"Error enabling InvokeModel logging: {e}")
        return False

def delete_bedrock_invokemodel_log_group(log_group_name):
    log_group_name = "/aws/bedrock/invokemodel"
    logs_client = boto3.client('logs')
    try:
        # First, disable the logging configuration in Bedrock
        bedrock_client = boto3.client('bedrock')
        bedrock_client.delete_model_invocation_logging_configuration()
        print("Successfully disabled InvokeModel logging for Bedrock")

        # Then, delete the log group
        logs_client.delete_log_group(logGroupName=log_group_name)
        print(f"Successfully deleted CloudWatch log group: {log_group_name}")
        return True
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print(f"Log group {log_group_name} does not exist.")
            return True
        else:
            print(f"Error deleting log group or disabling logging: {e}")
            return False


# Call the function
cw_log_group_name = "/aws/bedrock/invokemodel"

if enable_bedrock_invokemodel_logs(cw_log_group_name):
        print("Bedrock InvokeModel logging has been set up successfully.")
        log_group_name  = cw_log_group_name
else:
    print("Failed to set up Bedrock InvokeModel logging.")


print("Logroup name: ", log_group_name)

In [ ]:
import time
from datetime import datetime, timezone
from datetime import timedelta
import boto3
import json

cw_client = boto3.client('logs', region_name=region)

def query_model_invocation_log(query_string):
    # Start query
    startTime=int((datetime.now(timezone.utc) - timedelta(minutes=60)).timestamp())
    start_query_response = cw_client.start_query(
        logGroupName=log_group_name,
        startTime=int((datetime.now(timezone.utc) - timedelta(minutes=60)).timestamp()),
        endTime=int(datetime.now(timezone.utc).timestamp()),
        queryString=query_string,
    )
    query_id = start_query_response['queryId']
    # Wait for the query to complete
    response = None
    while response == None or response['status'] == 'Running':
        print('Waiting for query to complete ...')
        time.sleep(1)
        response = cw_client.get_query_results(
            queryId=query_id
        )

    # Print the results
    print(f"Query status: {response['status']}")
    return response

def print_filter_generation_output(user_query):
    print(user_query)
    # Construct CloudWatch Logs Insights query
    # This query:
    # 1. Selects timestamp and message fields
    # 2. Filters for filter generation task messages
    # 3. Matches the user's query
    # 4. Orders results by most recent first
    response = query_model_invocation_log(
        f"""
        fields @timestamp, @message
        | filter @message like /Your task is to structure the user's query to match the request schema provided below./
        | filter input.inputBodyJson.messages.0.content.0.text like /{user_query}/
        | sort @timestamp desc
        """)
    print(response)
    results = response['results']
    if (len(results) == 0):
        print("No results found")
        return
    
    result= results[0][1]['value']
    result_dict = json.loads(result)
    print(f"Generated filter:")
    filter_gen_output = result_dict['output']['outputBodyJson']['output']['message']['content'][0]['text']
    print(filter_gen_output)

### 2.5 Knowledge Base 실행 역할 업데이트

In [ ]:
# Before using autogenerated filters - update the knowledge base execution IAM role with right permissions

iam = boto3.resource('iam')
client = boto3.client('iam')

def get_attached_policies(role_name):
    response = client.list_attached_role_policies(RoleName=role_name)
    attached_policies = response['AttachedPolicies']
    return attached_policies

# get the knowledge base IAM role name
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb_id_metadata)
role_arn = get_kb_response['knowledgeBase']['roleArn']
role_name = role_arn.split('/')[-1]

# get attached policies
attached_policies = get_attached_policies(role_name)
attached_policies

def update_kb_execution_role(attached_policies, region_name):
    
    for policy in attached_policies:

        print(policy['PolicyArn'])
        policy_name = policy['PolicyName']
        policy_arn = policy['PolicyArn']

        if 'FoundationModel' in policy_arn:
            print('Updating FoundationModel policy: ',policy_arn)
            policy = iam.Policy(policy_arn)
            version = policy.default_version
            policyJson = version.document
            policyJson['Statement'][0]['Resource'].append('arn:aws:bedrock:{}::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'.format(region)) 
            policyJson['Statement'][0]['Resource'].append('arn:aws:bedrock:{}::foundation-model/anthropic.claude-3-5-sonnet-20240620-v1:0'.format(region))  
        
            client.detach_role_policy(RoleName=role_name,
                PolicyArn=policy_arn)
            
            response = client.delete_policy(
                PolicyArn=policy_arn
            )
            print(response)
           
            response = client.create_policy(
            PolicyName= policy_name,
            PolicyDocument=json.dumps(policyJson)
            )
            print(response)
        
        client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )

In [ ]:
update_kb_execution_role(attached_policies, region)
time.sleep(30)

### 2.6 메타데이터와 함께 Retrieve and Generate API로 Knowledge Base 질의(Autogenerated Filters 사용)

필터를 수동으로 만들지 않고, Amazon Bedrock Knowledge Bases가 자동으로 생성한 필터를 사용합니다.

In [ ]:
query = "How many prime members does Amazon have after 2021?"

#### 2.6.1 테스트 - Autogenerated Filters와 함께 RetrieveAndGenerate API 사용

먼저 샘플 사용자 질의를 바탕으로 `RetrieveAndGenerate` API가 자동 생성된 필터를 어떻게 처리하는지 살펴보겠습니다. `RetrieveAndGenerate` API는 Amazon Bedrock Knowledge Bases가 제공하는 API 중 하나입니다. 이 API는 유사도 검색을 기반으로 원하는 수만큼 문서 청크를 Knowledge Base에서 검색한 뒤, 검색된 청크와 LLM을 결합해 사용자 질문에 대한 답변을 생성합니다.

아래 코드에서는 특정 메타데이터 속성에 기반해 자동 생성된 필터를 사용하도록 `RetrieveAndGenerate` API를 구성합니다. 

`implicitFilterConfiguration`는 `RetrieveAndGenerate` API의 검색 과정에서 유사도 검색에 사용할 수 있는 메타데이터 속성을 정의하는 구성 객체입니다. 

이 구성에서는 어떤 메타데이터 필드를 필터링에 사용할 수 있는지 지정합니다. 여기서는 다음 세 가지 필드를 포함했습니다:

1. `year`: 문서가 다루는 연도를 나타내는 숫자입니다.
2. `company`: 문서가 설명하는 회사 이름을 나타내는 문자열입니다.
3. `ticker`: 해당 회사의 주식 티커 심볼을 나타내는 문자열입니다.

각 메타데이터 속성은 다음 속성을 가진 객체로 정의됩니다:

- `key`: 메타데이터 속성 이름입니다.
- `type`: 메타데이터 속성의 데이터 유형(예: NUMBER, STRING)입니다.
- `description`: 메타데이터 속성에 대한 설명으로, 가질 수 있는 값에 대한 정보를 포함합니다.

`implicitFilterConfiguration`는 또한 사용자 질의에 적합한 필터 표현식을 생성하는 데 사용할 Foundation Model의 ARN을 지정합니다. 여기서는 `anthropic.claude-3-5-sonnet-20240620-v1:0`으로 설정했습니다.

In [ ]:
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_metadata,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":10,
                    # "filter": { "equals": { "key": "x-amz-bedrock-kb-data-source-id", "value": ds_id }},
                    "implicitFilterConfiguration": {
                    "metadataAttributes":[
                        {
                            "key": "year",
                            "type": "NUMBER",
                            "description": "The year in which the document is about."
                        },
                        {
                            "key": "company",
                            "type": "STRING",
                            "description": "The company name the document is describing. Possible values include ['Amazon']"
                        },
                        {
                            "key": "ticker",
                            "type": "STRING",
                            "description": "The ticker name of the company. Possible values include ['AMZN']"
                        }
                    ],
                    "modelArn": "arn:aws:bedrock:{}::foundation-model/anthropic.claude-3-5-sonnet-20240620-v1:0".format(region)
                },
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

이제 주어진 사용자 질의에 대해 생성된 자동 필터를 확인하기 위해 CloudWatch 로그를 살펴보겠습니다.

Autogenerated Filter 구성을 가진 `RetrieveAndGenerate` API는 사용자 질의를 처리하고 메타데이터 속성을 기반으로 구조화된 필터 표현식을 생성합니다. 이 필터 표현식은 논리 연산자(`and`, `or`)와 비교 연산자(`eq`, `gt`, `lt` 등)를 포함하는 JSON 객체로 표현됩니다. 비교 연산자는 메타데이터 속성과 해당 값을 지정해 문서를 필터링합니다.

모델은 사용자 질의와 구성된 메타데이터 속성을 분석해 필터 표현식을 구성하는 논리 연산과 값을 결정합니다. 질의에서 언급된 관련 메타데이터 속성과 값을 식별한 다음, 적절한 논리 연산자와 비교 연산자를 사용해 필터 표현식을 만듭니다.

In [ ]:
# check cloudwatch logs for the implict filter generated
time.sleep(60)
print_filter_generation_output(query)

#### 2.6.2 테스트 - Autogenerated Filters와 함께 Retrieve API 사용

다음으로 `Retrieve` API에서 자동 생성된 필터 구성을 시험해 보겠습니다. `Retrieve` API는 Amazon Bedrock Knowledge Bases가 제공하는 또 다른 API로, 사용자 질의를 임베딩으로 변환하고 Knowledge Base를 검색하여 관련 결과를 반환하며, 의미 기반 검색 결과 위에 사용자 정의 워크플로를 구축할 수 있도록 더 많은 제어 권한을 제공합니다. Retrieve API의 출력에는 검색된 텍스트 청크, 소스 데이터의 위치 유형과 URI, 검색 점수가 포함됩니다.

`Retrieve` API가 반환하는 일부 청크에는 자동으로 생성된 필터를 기반으로 한 메타데이터가 함께 포함되어 있음을 확인할 수 있습니다. 

1. 관련 연도: 청크의 `year` 메타데이터가 2021보다 큰 값으로 설정되어 있어, "2021년 이후 Amazon의 Prime 회원"에 대한 질의와 일치합니다.

2. 회사 정보: 메타데이터의 `company` 필드가 일관되게 "Amazon"으로 표시되어 특정 회사에 대한 질의와 맞습니다.

3. 주식 티커 정보: `ticker` 필드가 일관되게 "AMZN"으로 표시되어 특정 주식에 대한 질의와 맞습니다.

이 예시는 명시적으로 필터를 지정하지 않아도 자동 생성된 필터가 질의에 내포된 조건을 기반으로 검색 결과를 효과적으로 좁히는 방식을 보여 줍니다. 시스템은 2023년 매출에 대한 사용자 질의를 해석하고 Knowledge Base에서 가장 관련성이 높은 정보를 반환하도록 자동으로 필터를 적용했습니다.

In [ ]:
response_ret_with_implicit_fiters = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id_metadata, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":10,
            "implicitFilterConfiguration": {
                    "metadataAttributes":[
                        {
                            "key": "year",
                            "type": "NUMBER",
                            "description": "The year in which the document is about."
                        },
                        {
                            "key": "company",
                            "type": "STRING",
                            "description": "The company name the document is describing. Possible values include ['Amazon']"
                        },
                        {
                            "key": "ticker",
                            "type": "STRING",
                            "description": "The ticker name of the company. Possible values include ['AMZN']"
                        }
                    ],
                    "modelArn": "arn:aws:bedrock:{}::foundation-model/anthropic.claude-3-5-sonnet-20240620-v1:0".format(region)
                },
        } 
    },
    retrievalQuery={
        "text": query
    }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(retrieve_resp['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret_with_implicit_fiters)

In [ ]:
# check cloudwatch logs for the implict filter generated
time.sleep(20)
print_filter_generation_output(query)

### 2.7 정리
이 노트북에서 만든 리소스를 삭제하려면 아래 셀의 주석을 해제한 뒤 실행하세요.

In [ ]:
# delete local directory
import shutil

dir_path = "sec-10-k" # Replace with the actual path

try:
    shutil.rmtree(dir_path)
    print(f"Directory '{dir_path}' and its contents have been deleted successfully.")
except FileNotFoundError:
    print(f"Directory '{dir_path}' not found.")
except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
# Delete log group name
delete_bedrock_invokemodel_log_group(log_group_name)

In [ ]:
## Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

In [ ]:
print("===============================Knowledge base==============================")
knowledge_base_metadata.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)